In [1]:
import pandas as pd 
import math, random
all_data  = pd.read_csv("sensor_data_600.txt", delimiter=" ", header=None, names = ("date","time","ir","z"))#lidarのセンサ値は「z」に
data = all_data.sample(3000).sort_values(by="z").reset_index()  #1000個だけサンプリングしてインデックスを振り直す
data = pd.DataFrame(data["z"])

In [2]:
##負担率の初期化## 

K = 3 #クラスタ数
n = int(math.ceil(len(data)/K)) #クラスタあたりのセンサ値の数
for k in range(K):
    data[k] = [1.0 if k == int(i/n) else 0.0 for i,d in data.iterrows()] #データをK個に分けて、一つのr_{i,k}を1に。他を0に。

In [3]:
def update_parameters(ds, k, mu_avg=600, zeta=1, alpha=1, beta=1, tau=1): 
    R = sum([d[k] for _, d in ds.iterrows()])
    S = sum([d[k]*d["z"] for _, d in ds.iterrows()])
    T = sum([d[k]*(d["z"]**2) for _, d in ds.iterrows()])
    
    hat = {}

    hat["tau"] = R + tau
    hat["zeta"] = R + zeta
    hat["mu_avg"] = (S + zeta*mu_avg)/hat["zeta"]
    hat["alpha"] = R/2 + alpha
    hat["beta"] = (T + zeta*(mu_avg**2) - hat["zeta"]*(hat["mu_avg"]**2))/2 + beta
    
    hat["z_std"] = math.sqrt(hat["beta"]/hat["alpha"])
    
    return pd.DataFrame(hat, index=[k])

In [4]:
from scipy.stats import norm, dirichlet
import matplotlib.pyplot as plt
import numpy as np

def draw(ps):
    pi = dirichlet([ps["tau"][k] for k in range(K)]).rvs()[0]
    pdfs = [ norm(loc=ps["mu_avg"][k], scale=ps["z_std"][k]) for k in range(K) ]

    xs = np.arange(600,650,0.5)

    ##p(z)の描画##
    ys = [ sum([pdfs[k].pdf(x)*pi[k] for k in range(K)])*len(data) for x in xs] #pdfを足してデータ数をかける
    plt.plot(xs, ys, color="red")

    ##各ガウス分布の描画##
    for k in range(K):
        ys = [pdfs[k].pdf(x)*pi[k]*len(data) for x in xs]
        plt.plot(xs, ys, color="blue")

    ##元のデータのヒストグラムの描画##
    data["z"].hist(bins = max(data["z"]) - min(data["z"]), align='left', alpha=0.4, color="gray")
    plt.show()

In [5]:
from scipy.special import digamma 

def responsibility(z, K, ps):
    tau_sum = sum([ps["tau"][k] for k in range(K)])
    r = {}
    for k in range(K):
        log_rho = (digamma(ps["alpha"][k]) - math.log(ps["beta"][k]))/2 \
                            - (1/ps["zeta"][k] + ((ps["mu_avg"][k] - z)**2)*ps["alpha"][k]/ps["beta"][k])/2 \
                            + digamma(ps["tau"][k]) - digamma(tau_sum)
                
        r[k] = math.exp(log_rho)
       
    w = sum([ r[k] for k in range(K) ]) #正規化
    for k in range(K): r[k] /= w
    
    return r

In [6]:
def one_step(ds): ###variationalinference2onestep
    ##パラメータの更新##
    params = pd.concat([update_parameters(ds, k) for k in range(K)]) 

    ##負担率の更新##
    rs = [responsibility(d["z"], K, params) for _, d in ds.iterrows() ]
    for k in range(K):
        ds[k] = [rs[i][k] for i,_ in data.iterrows()]
        
    return ds, params

In [7]:
params_history = {} ###variationalinference2iter
for t in range(1, 10000):
    data, params = one_step(data)
    if t%10 ==0:              #10回ごとにパラメータを記録
        params_history[t] = params
        display(params)
        
        if len(params_history) < 2:
            continue
           
        if all([ abs(params_history[t-10]["tau"][k] - params_history[t]["tau"][k]) < 10e-5 for k in range(K)]):
            break

,tau,zeta,mu_avg,alpha,beta,z_std
0,1.312760,1.312760,604.869825,1.156380,55.122629,6.904221
1,5.670545,5.670545,620.197578,3.335273,347.961031,10.214089
2,9.955513,9.955513,622.292816,5.477757,459.140166,9.155273
3,13.539574,13.539574,623.116492,7.269787,537.542405,8.598952
4,17.220424,17.220424,623.775173,9.110212,617.988465,8.236182
5,21.407546,21.407546,624.402453,11.203773,709.042270,7.955251
6,23.962853,23.962853,624.723274,12.481427,762.974418,7.818490
7,30.161668,30.161668,625.308665,15.580834,885.645791,7.539363
8,37.042553,37.042553,625.777323,19.021276,1012.541898,7.296031
9,44.913426,44.913426,626.188279,22.956713,1149.112915,7.075001


,tau,zeta,mu_avg,alpha,beta,z_std
0,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
3,1.079164,1.079164,601.278707,1.039582,12.646903,3.487890
4,2.068750,2.068750,611.836105,1.534375,158.789538,10.172909
5,4.838667,4.838667,619.557417,2.919333,329.108104,10.617626
6,7.338467,7.338467,621.695089,4.169234,415.537404,9.983365
7,15.133396,15.133396,624.069902,8.066698,612.203886,8.711645
8,25.550016,25.550016,625.196917,13.275008,827.037603,7.893057
9,38.795802,38.795802,625.916301,19.897901,1071.147501,7.337042


,tau,zeta,mu_avg,alpha,beta,z_std
0,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
5,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
6,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
7,1.329975,1.329975,604.991170,1.164987,55.771783,6.919053
8,8.059212,8.059212,621.930142,4.529606,434.632490,9.795597
9,24.479944,24.479944,624.953354,12.739972,809.403335,7.970733


,tau,zeta,mu_avg,alpha,beta,z_std
0,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
5,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
6,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
7,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
8,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
9,6.618817,6.618817,620.957475,3.809409,388.156255,10.094261


,tau,zeta,mu_avg,alpha,beta,z_std
0,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
5,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
6,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
7,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
8,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
9,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000


...

,tau,zeta,mu_avg,alpha,beta,z_std
0,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
5,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
6,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
7,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
8,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
9,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000


,tau,zeta,mu_avg,alpha,beta,z_std
0,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
5,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
6,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
7,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
8,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
9,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000


,tau,zeta,mu_avg,alpha,beta,z_std
0,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
5,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
6,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
7,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
8,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
9,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000


,tau,zeta,mu_avg,alpha,beta,z_std
0,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
5,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
6,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
7,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
8,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000
9,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000


KeyboardInterrupt: 

In [ ]:
draw(params) ###variationalinference2draw

In [ ]:
draw(params_history[100])

In [ ]:
draw(params_history[10])

In [ ]:
params_history